# IQ Noise Detection Inference
This Notebook assumes post pass JSON reports of IQ Constellation plots
1. Apply KMeans Clustering to map blobs to clusters
2. Extract Blob Shape by applying Covariance Error Ellipse
3. Use statistical methods to determine Interference/Phase Noise/Compression/IQ Imablance/Normal features
4. Invoke model endpoint which features to infer whether noise is present in the signal
5. Write noise detection notifications to S3 to alert downstream applications

Kernel Used: python3, ml.m5.xlarge

## Imports

In [1]:
import numpy as np
import pandas as pd
import os
import time
import datetime
import json
import boto3
from autogluon.tabular import TabularPredictor

from utility import constellation_metrics

## Parameters

In [7]:
model_path = "MODEL_PATH" # See output of IQ-data-train-classifier.ipynb
bucket_name = "BUCKET_NAME" # Bucket to output inference insights

In [8]:
# Setup S3 client
s3 = boto3.client('s3')

## Setup inference to test against 32apsk samples in the inference/ directory

In [9]:
modcod = '32apsk'
base_path = './inference/'

## Process IQ Constellation plots

In [10]:
# List to store all results
all_results = []
filenames = []  # New list to store filenames    

processor = constellation_metrics.ConstellationProcessor(modcod=modcod)

if not os.path.exists(base_path):
    print(f"    Warning: Path {base_path} does not exist. Skipping...")
    exit(-1)

# Process each file in directory
for filename in os.listdir(base_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(base_path, filename)
        result = processor.process_file(file_path, 'n/a', plot=False)
        if result:
            all_results.append(result)
            filenames.append(filename)  # Store the filename

# Convert to DataFrame
results_df = pd.DataFrame(all_results)

## Setup Infernce endpoint and infer noise classes

In [11]:
predictor = TabularPredictor.load(model_path)
predictions = []
# run predict for each in results_df
for index, row in results_df.iterrows():
    # drop class label
    row = row.drop('class')
    # convert to dataframe
    row = pd.DataFrame([row])
    # run predict
    predictions.append({"class": predictor.predict(row).iloc[0], "fileId": filenames[index]})

In [ ]:
predictions

## Write notifications to S3 if noise classes are present

In [ ]:
# Export Results
write_buffer = []
for j in predictions:
    if (j['class'] != "normal"):
        mydata = {"class": j['class'], "timestamp": time.time(), "fileId": j['fileId']}
        write_buffer.append(mydata)

if (len(write_buffer) > 0):
    sttime = datetime.datetime.fromtimestamp(time.time()).strftime('%Y%m%d%H%M%S.json')
    print("Writting results to S3")
    print(write_buffer)
    # Write output to S3
    s3.put_object(Body=json.dumps(write_buffer), Bucket=bucket_name, Key='digital_rf_detection/'+sttime)